In [1]:
%reload_ext autoreload
%autoreload 2

from llm_blocks import blocks

In [11]:
template = """You're a sophisticated software development AI expert system, capable of assistance with the development of advanced software applications. Your job is to produce comprehensive software architecture designs for MVP software solutions.

One of your clients is requesting a design for the following application:
{application_description}

Be concise, but include all the necessary details."""

In [12]:
block = blocks.Block(template=template, stream=True)
block(application_description="AI assisted meal planning & grocery list given nutritional goals and dietary restrictions.")

Title: AI Assisted Meal Planning & Grocery List Application

Overview:
The AI Assisted Meal Planning & Grocery List application aims to provide users with personalized meal plans and grocery lists based on their nutritional goals and dietary restrictions. By leveraging artificial intelligence algorithms, the application will analyze user preferences, dietary requirements, and health goals to generate optimized meal plans and corresponding grocery lists.

Key Components:
1. User Interface (UI):
   - User Registration and Authentication: Allow users to create accounts and securely log in to access personalized features.
   - Profile Management: Enable users to set their nutritional goals, dietary restrictions, and preferences.
   - Meal Plan Visualization: Display personalized meal plans with details such as recipes, ingredients, and nutritional information.
   - Grocery List Generation: Generate a comprehensive grocery list based on the selected meal plan, including quantities and categ